## Prerequisites

In [1]:
!pip install numpy matplotlib pandas sklearn

In [2]:
!python -V

Python 3.8.8


## Data Preprocessing

In [3]:
import scipy

In [4]:
from scipy.io import loadmat

In [5]:
import pickle

In [8]:
data = loadmat(r'Al-Sa-weidianzi-generateTTRnotshift.mat')

data = scipy.io.loadmat('re20220415.mat')

In [9]:
print(data['re'])

[[0.51874289 0.55444985 0.59019769 ... 0.0166075  0.01644659 0.01628653]
 [0.51880569 0.55451583 0.59026658 ... 0.01648318 0.01632349 0.01616412]
 [0.51886721 0.55458046 0.59033406 ... 0.01636108 0.01620214 0.01604406]
 ...
 [0.74210795 0.77924699 0.81446449 ... 0.02428921 0.02404812 0.02380834]
 [0.74243714 0.77956991 0.81477718 ... 0.02423477 0.0239936  0.02375427]
 [0.74276444 0.77989096 0.81508806 ... 0.02418054 0.02393981 0.02370079]]


In [10]:
import pickle
import numpy as np

In [11]:
np.set_printoptions(suppress=True)

In [12]:
X = np.transpose(data['re'])

In [13]:
X.shape

(300, 10000)

In [14]:
X = X.T

In [15]:
X.shape

(10000, 300)

In [18]:
layer_1 = np.linspace(250, 500, 100, endpoint=True)
layer_4 = np.linspace(0.1, 1.5, 100, endpoint=True)
layer_2 = layer_4*100

In [19]:
y = np.zeros((X.shape[0], 2))

In [20]:
i = 0
for l2 in layer_2:
    for l1 in layer_1:
            y[i] = [ l2, l1]
            i = i + 1

In [21]:
y.shape

(10000, 2)

### Train test split

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0)

In [25]:
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split

In [26]:
#property = 0

In [27]:
x_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float()

x_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).float()

In [28]:
# Builds dataset with ALL data
origin_train_dataset = TensorDataset(x_train_tensor, y_train_tensor)

# Splits randomly into train and validation datasets
train_dataset, val_dataset = random_split(origin_train_dataset, [int(x_train_tensor.shape[0] * 0.9), int(x_train_tensor.shape[0] * 0.1)])

# Builds a loader for each dataset to perform mini-batch gradient descent
train_loader = DataLoader(dataset=train_dataset, batch_size=2000)
val_loader = DataLoader(dataset=val_dataset, batch_size=2000)

test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader  = DataLoader(dataset=test_dataset, batch_size=2000)

## Model

In [29]:
import torch.nn as nn

In [30]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.bn1 = nn.BatchNorm1d(X.shape[1])
        self.fc1 = nn.Linear(X.shape[1], 100)
        self.bn2 = nn.BatchNorm1d(100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 10)
        self.fc4 = nn.Linear(10, 2)

    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = torch.tanh(x)
        x = self.bn2(x)
        x = self.fc2(x)
        x = torch.tanh(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.fc4(x)
        x = torch.relu(x)
        return x

## Training

In [31]:
import torch.optim as optim

In [32]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [33]:
n_epochs = 250

In [34]:
def make_train_step(model, loss_fn, optimizer):
    def train_step(x, y):
        model.train()
        yh = model(x)
        #yh = torch.reshape(yh, (-1,))
        loss = loss_fn(y, yh)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return train_step

In [35]:
model = Net().to(device)

loss_fn = nn.MSELoss(reduction='mean')

# optimizer = optim.SGD(model.parameters(), lr=0.01)
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)

train_step = make_train_step(model, loss_fn, optimizer)

In [36]:
model.eval()

Net(
  (bn1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=300, out_features=100, bias=True)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=2, bias=True)
)

In [37]:
training_losses = []
validation_losses = []

for epoch in range(n_epochs):
    batch_losses = []
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        loss = train_step(x_batch, y_batch)
        batch_losses.append(loss)
    training_loss = np.mean(batch_losses)
    training_losses.append(training_loss)

    with torch.no_grad():
        val_losses = []
        for x_val, y_val in val_loader:
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            model.eval()
            yh = model(x_val)
            #yh = torch.reshape(yh, (-1,))
            val_loss = loss_fn(y_val, yh).item()
            val_losses.append(val_loss)
        validation_loss = np.mean(val_losses)
        validation_losses.append(validation_loss)

    print(f"[{epoch+1}] Training loss: {training_loss:.5f}\t Validation loss: {validation_loss:.5f}")

[1] Training loss: 667.19224	 Validation loss: 680.97479
[2] Training loss: 637.11033	 Validation loss: 657.40625
[3] Training loss: 599.71395	 Validation loss: 607.15778
[4] Training loss: 554.35172	 Validation loss: 544.28241
[5] Training loss: 502.93427	 Validation loss: 484.82651
[6] Training loss: 448.33002	 Validation loss: 428.24625
[7] Training loss: 393.58810	 Validation loss: 379.67892
[8] Training loss: 340.43217	 Validation loss: 325.32086
[9] Training loss: 292.26480	 Validation loss: 281.50204
[10] Training loss: 245.66990	 Validation loss: 231.21095
[11] Training loss: 201.21704	 Validation loss: 190.45624
[12] Training loss: 160.51183	 Validation loss: 140.23630
[13] Training loss: 125.75201	 Validation loss: 111.24151
[14] Training loss: 92.49930	 Validation loss: 74.91708
[15] Training loss: 73.49213	 Validation loss: 62.28900
[16] Training loss: 56.45134	 Validation loss: 50.37185
[17] Training loss: 50.95397	 Validation loss: 45.55236
[18] Training loss: 44.31198	 V

[152] Training loss: 0.34051	 Validation loss: 0.14648
[153] Training loss: 0.50300	 Validation loss: 0.48502
[154] Training loss: 0.42649	 Validation loss: 1.92804
[155] Training loss: 0.54733	 Validation loss: 0.90034
[156] Training loss: 0.43373	 Validation loss: 0.24133
[157] Training loss: 0.52626	 Validation loss: 0.68039
[158] Training loss: 0.43917	 Validation loss: 0.45761
[159] Training loss: 0.31187	 Validation loss: 0.49063
[160] Training loss: 0.38038	 Validation loss: 0.35886
[161] Training loss: 0.36370	 Validation loss: 0.72626
[162] Training loss: 0.35470	 Validation loss: 0.54933
[163] Training loss: 0.31655	 Validation loss: 0.33894
[164] Training loss: 0.30605	 Validation loss: 0.14047
[165] Training loss: 0.49062	 Validation loss: 0.84296
[166] Training loss: 0.65160	 Validation loss: 0.44456
[167] Training loss: 0.35332	 Validation loss: 0.44865
[168] Training loss: 0.41996	 Validation loss: 0.28210
[169] Training loss: 0.21726	 Validation loss: 0.07942
[170] Trai

In [38]:
# model.state_dict()

## Testing

In [39]:
def mean_absolute_percentage_error(y_true, y_pred):
    return torch.mean(torch.abs((y_true - y_pred) / y_true)) * 100

In [40]:
x_test_tensor = x_test_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)
y_pred = model(x_test_tensor).squeeze()

In [41]:
test_loss = loss_fn(y_test_tensor, y_pred)
print(test_loss)

tensor(0.3599, grad_fn=<MseLossBackward0>)


In [42]:
print(f"The mean of absolute percentage error: {mean_absolute_percentage_error(y_test_tensor.cpu(), y_pred.cpu()):.2f}%")

The mean of absolute percentage error for L1: 2.30%


## Real-world case predict

In [45]:
import pandas as pd

In [46]:
case1 = pd.read_csv('Al-Sa-weidianzi.csv', header=None)

In [47]:
case1 = np.reshape(np.array(case1), [ 1,-1])

In [48]:
case1.shape

(1, 300)

In [49]:
case1= np.array(case1)

In [51]:
case1_tensor = torch.from_numpy(case1).float()

In [52]:
model.eval()
pred_1 = model(case1_tensor)

In [53]:
pred_1

tensor([[ 5.0304, 32.6180]], grad_fn=<ReluBackward0>)